In [1]:
import jellyfish
import time
import pickle

def get_list_of_words(file_path):
    words = []
    with open(file_path, 'r') as f:
        for line in f:
            word = line.strip().lower()
            words.append(word)
    return words

# hamming / jaro_winkler(s1,s2) .... not suitable we want similar sound
# soundex(s1)            <-- 1900 ??
# nysiis(s1)             <-- 1970
# match_rating_codex(s1) <-- 1977 MRA
# metaphone(s1)          <-- 1990  [double metaphone - postgre]
# match_rating_comparison(s1,s2) -- True/False

def compare_words(word_list, threshold=0.80, total_len=235886):
    word_dict = {}
    start = time.process_time()

    for i, w1 in enumerate(word_list):
        if i > 0 and (i+1) % 100 == 0:
            t = time.process_time() - start
            start = time.process_time()
            print("{}/{} .. {:.1f}".format(i+1, total_len, t))
        similar = []
        
        # comparte w1 to all other words
        for w2 in word_list:
            if w1 == w2: # skip same
                continue
            elif jellyfish.match_rating_comparison(w1, w2): # must be phonetically similar
                    score = jellyfish.jaro_winkler(w1,w2)
                    if score > threshold:  # and score must be higher than threshold
                        similar.append((w2, score))
                
        # append only if we have some similar words...
        if len(similar) > 0:
            # sort by score ...
            similar = sorted(similar, key=lambda tup: tup[1], reverse=True)
            word_dict[w1] = similar
                
    return word_dict

def search(data, word, count=10):
    all_homz = data[word]
    output = []
    if len(all_homz) > 0:
        for h in all_homz:
            output.append(h[0])
            if len(output) >= count:
                return output

def save_data(obj, name):
    with open('data/' + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_data(name):
    with open('data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [2]:
words  = get_list_of_words('english-words/english_words.txt') # 1529
result = compare_words(words)

KeyboardInterrupt: 

In [37]:
#result

{'a': [('ax', 0.8333333333333334),
  ('ad', 0.8333333333333334),
  ('an', 0.8333333333333334)],
 'acts': [('facts', 0.9333333333333332),
  ('cast', 0.8333333333333334),
  ('pact', 0.8333333333333334),
  ('wacs', 0.8333333333333334),
  ('chants', 0.8055555555555555)],
 'ax': [('a', 0.8333333333333334)],
 'ad': [('add', 0.8888888888888888),
  ('ads', 0.8888888888888888),
  ('ade', 0.8888888888888888),
  ('a', 0.8333333333333334),
  ('adds', 0.8333333333333334),
  ('adze', 0.8333333333333334),
  ('aide', 0.8333333333333334),
  ('made', 0.8333333333333334),
  ('wade', 0.8333333333333334)],
 'add': [('adds', 0.9166666666666666), ('ad', 0.8888888888888888)],
 'adds': [('add', 0.9166666666666666),
  ('ads', 0.9166666666666666),
  ('ad', 0.8333333333333334)],
 'ads': [('adds', 0.9166666666666666),
  ('ad', 0.8888888888888888),
  ('days', 0.8055555555555555)],
 'adze': [('ade', 0.9166666666666666),
  ('daze', 0.9166666666666666),
  ('aide', 0.8500000000000001),
  ('ad', 0.8333333333333334),
  (

In [ ]:
 # have 